In [28]:
import os
import pandas as pd
import re

events = [os.path.join(route, f) for route, dirs, files in os.walk('../bids') for f in files if f.endswith('events.tsv')]

In [31]:
df_list = []
for f in events:
    temp = pd.read_csv(f,sep='\t')
    sub = re.search('sub-(.*)/func', f).group(1)
    acq = re.search('acq-(.*)_events', f).group(1)
    temp['sub'] = sub
    temp['acq'] = acq
    df_list.append(temp)
df = pd.concat(df_list)
df = df[['sub','acq','duration','trial_type','response_time']]
df = df[df['trial_type'].str.contains('outcome') == False]
df[['phase','button','partner']] = df['trial_type'].str.split('_',2, expand = True)
df['button'] = df['button'].map({'rightButton':-1, 'leftButton':1})
df = df.groupby(by = ['sub','acq','partner']).mean()
df.to_csv('../derivatives/mean_buttonpress.csv')